In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Authenticate Spotify API

In [9]:
# Authenticating....

clientID = '76f800b58144403a88a52b67eb7708ed'
clientSecret = '7a560acfb8cc4f43b28766bc1683d822'
redirect_uri = 'http://localhost:8888'
username = "ashiquemahmood"


scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username
                                   , scope,client_id=clientID,
                                  client_secret=clientSecret,
                                  redirect_uri=redirect_uri)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

### Get playlist IDs

In [3]:
playlists = sp.user_playlists(username)

In [4]:
for playlist in playlists['items']:
    if playlist['name'] == 'like':
        print(playlist['id'])
        likeId = playlist['id']
    if playlist['name'] == 'dislike':
        print(playlist['id'])
        dislikeId = playlist['id']
    

5sq1Dvla1U7a8LGFKBduLn
4ESdFDYte6M94ToYGMfWYb


### Get Audio Features for songs we like

In [5]:
def get_audio_features(playlist,label):
    songIds = []
    songNames = []
    songFeatures = []
    
    tracks = playlist['tracks']
    songs = tracks['items']
    
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks['items']:
            songs.append(item)
    
    for song in songs:
        songIds.append(song['track']['id'])
        songNames.append(song['track']['name'])
    
    for i in range(0,len(songs),50):
        features = sp.audio_features(songIds[i:50+i])
        for feature in features:
            songFeatures.append(feature)
    
    
    data = pd.DataFrame(songFeatures)
    data['name'] = songNames
    data['target'] = label
    
    return data

In [6]:
like = sp.user_playlist(username,likeId)
likeSongsFeatures = get_audio_features(like,1)

### Get Audio Features for songs we don't like

In [7]:
dislike = sp.user_playlist(username,dislikeId)
dislikeSongsFeatures = get_audio_features(dislike,0)

### Data Exploration

In [8]:
likeSongsFeatures.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
count,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.0
mean,0.144025,0.54513,259286.075000,0.719670,0.134014,5.215000,0.194011,-6.609990,0.670000,0.054604,122.464360,3.980000,0.479473,1.0
std,0.211763,0.14111,77052.496259,0.181679,0.243787,3.809374,0.168066,2.745341,0.471393,0.044299,26.426114,0.172478,0.220325,0.0
min,0.000009,0.14500,96560.000000,0.127000,0.000000,0.000000,0.037500,-17.260000,0.000000,0.023200,75.305000,3.000000,0.051700,1.0
25%,0.006217,0.44775,215393.250000,0.607000,0.000010,1.000000,0.093625,-7.893250,0.000000,0.033000,101.479750,4.000000,0.317000,1.0
50%,0.044850,0.54850,237966.500000,0.760500,0.002145,5.000000,0.123500,-6.194000,1.000000,0.040500,120.938000,4.000000,0.471500,1.0
75%,0.199250,0.63600,284990.250000,0.857000,0.137750,9.000000,0.228250,-4.591250,1.000000,0.058375,143.235500,4.000000,0.635500,1.0
max,0.978000,0.86300,608280.000000,0.988000,0.910000,11.000000,0.912000,-1.832000,1.000000,0.347000,186.054000,5.000000,0.964000,1.0
